# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
# import prerequisite python modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, log_loss, roc_auc_score


In [2]:
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [3]:

X = adult_dt[columns[:-1]]
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*From the documentation linked:*

>#### Random State
>Controls the shuffling applied to the data before applying the split. Pass an int for reproducible output across multiple function calls.

*The integer held in random state fixes the seed for the pseudo-random algorithm causing the it to similarly fix its output. This is useful for sharing and for verifying code changes while analysis is in development, but in a production settings the random state should not be a fixed value*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [4]:
adult_dt.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [5]:
adult_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 3.7+ MB


In [6]:
#define numeric preprocessing with a list of features and an sklearn.Pipeline
ft_num = ['age',
         'fnlwgt',
         'education-num',
         'capital-gain',
         'capital-loss',
         'hours-per-week',
         ]
pipe_num = Pipeline([
    ('num_impute',KNNImputer(n_neighbors=7,weights='distance')),
    ('num_preprocessing',RobustScaler()),
])

#define categoric preprocessing with a list of features and an sklearn.Pipeline
ft_cat = ['workclass',
         'education',
         'marital-status',
         'occupation',
         'relationship',
         'race',
         'sex',
         'native-country'
         ]
pipe_cat = Pipeline([
    ('cat_impute',SimpleImputer(strategy = 'most_frequent')),
    ('cat_preprocessing',OneHotEncoder(categories='auto',drop='if_binary')),
])

#define preprocessing as sklearn.ColumnTransformer
column_transformer = ColumnTransformer([
    ('num_transforms',pipe_num,ft_num),
    ('cat_transforms',pipe_cat,ft_cat),
    ])

#Verify the sklearn objects match the brief by calling them
#pipe_cat
#pipe_num
column_transformer

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('num_impute',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('num_preprocessing',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('cat_impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('cat_preprocessing',
                                                  OneHotEncoder(drop='if_binary'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [7]:

pipe_numcat = Pipeline([
    ('preprocessing',column_transformer),
    ('model',RandomForestClassifier(criterion='entropy'))
])
pipe_numcat

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('num_impute',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('num_preprocessing',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('cat_preprocessing',
                                                                   OneHotEncoder(drop='if_binary'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('model', RandomForestClassifier(criterion='entropy'))])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [8]:
performance_metrics = [
    'neg_log_loss',
    'roc_auc',
    'accuracy',
    'balanced_accuracy',
]

res_simple_dict = cross_validate(pipe_numcat, X_train, Y_train, cv=5, scoring=performance_metrics)
res_simple = pd.DataFrame(res_simple_dict)
res_simple

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 139, in __call__
    score = scorer._score(
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/utils/_response.py", line 211, in _get_response_values
    y_pred = prediction_method(X)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/pipeline.py", line 

,fit_time,score_time,test_neg_log_loss,test_roc_auc,test_accuracy,test_balanced_accuracy
0,2.698280,0.020888,NaN,NaN,NaN,NaN
1,2.693555,0.100537,-0.358903,0.903600,0.848212,0.768590
2,2.690063,0.098896,-0.376427,0.902038,0.852567,0.772542
3,2.754719,0.099918,-0.371345,0.906622,0.859588,0.783021
4,2.773974,0.102376,-0.369261,0.905239,0.854541,0.773230


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [9]:
res_simple.sort_values(by='test_neg_log_loss')

,fit_time,score_time,test_neg_log_loss,test_roc_auc,test_accuracy,test_balanced_accuracy
2,2.690063,0.098896,-0.376427,0.902038,0.852567,0.772542
3,2.754719,0.099918,-0.371345,0.906622,0.859588,0.783021
4,2.773974,0.102376,-0.369261,0.905239,0.854541,0.773230
1,2.693555,0.100537,-0.358903,0.903600,0.848212,0.768590
0,2.698280,0.020888,NaN,NaN,NaN,NaN


Calculate the mean of each metric. 

In [10]:
#res_simple.describe()
res_simple.mean()

fit_time                  2.722118
score_time                0.084523
test_neg_log_loss        -0.368984
test_roc_auc              0.904375
test_accuracy             0.853727
test_balanced_accuracy    0.774346
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [11]:
pipe_numcat.fit(X_train, Y_train)

#Y_pred_train = pipe_numcat.predict(X_train)
Y_pred_test = pipe_numcat.predict(X_test)

#Y_proba_train = pipe_numcat.predict_proba(X_train)
Y_proba_test = pipe_numcat.predict_proba(X_test)


In [12]:
results = {
    'accuracy_score_test': accuracy_score(Y_test, Y_pred_test),
    'roc_auc_test': roc_auc_score(Y_test, Y_proba_test[:,1]),
    'neg_log_loss_test': log_loss(Y_test, Y_proba_test[:,1])*(-1),
    'balanced_accuracy_test': balanced_accuracy_score(Y_test, Y_pred_test)
}
results

{'accuracy_score_test': 0.853618589415498,
 'roc_auc_test': 0.8998344141483878,
 'neg_log_loss_test': -0.3908354852764272,
 'balanced_accuracy_test': 0.7732630876007993}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

>Why did we manually encode the target variable during the raw data read? The preprocessing performed in our pipeline is applied to features and not the target. Hence we need to choose a timely step to encode the target variable. 

>A poor choice of step to encode the target might be when we divide the raw data into an X dataframe holding features and Y dataframe holding the target. In that case we would have an unexplained hesitation to preprocess the features along with the target. 

>Instead, encoding the target while loading the raw_data allows us to ensure the placement (last column) and efficient encoding (as 1 or 0) which makes the X,Y split step highly readable

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.